In [ ]:
# !pip install lxml

In [ ]:
#import sys
#print(sys.executable)

In [ ]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import numpy as np

In [ ]:
webpage=requests.get('https://www.eatright.org/recipes/breakfast').text

In [ ]:
soup = BeautifulSoup(webpage,'lxml')

In [ ]:
#print(soup.prettify())   ----->>> use to arrange the messed up code in decent coding pattern for easy understandability.

In [ ]:
soup.find(class_='article-listing__grid').text

In [ ]:
articles=soup.find_all("article",class_='card card--clickable')

In [ ]:
print(len(articles))
print(type(articles))

In [ ]:
print(articles[0].h3.text)

In [ ]:
articles[0].a.get('href')

In [ ]:
breakfast={}
for i in articles:
    breakfast[i.h3.text]=i.a.get('href')
print(breakfast[articles[0].h3.text])

In [ ]:
#    https://www.eatright.org

In [ ]:
i=articles[2].a.get('href')
url="https://www.eatright.org{}".format(i)
response = requests.get(url).text

In [ ]:
print(url)

In [ ]:
soup2=BeautifulSoup(response,"lxml")
section=soup2.find("section",class_="rtf rtf--with-margin")

In [ ]:
for tag in section:
    if tag.name=="h3":
        print("---")
    else:
        print(tag)

In [ ]:
description = []
ingredients = []
direction = []
nutrition = []

i = 0   # important: before first h3

for tag in section:

    if tag.name == "h3":
        i += 1
        continue

    if tag.name == "p":

        text = tag.get_text(strip=True)

        if i == 0:
            description.append(text)
        elif i == 1:
            ingredients.append(text)
        # elif i == 2:
            #direction.append(text)
        elif i == 3:
            nutrition.append(text)


In [ ]:
description = []
ingredients = []
nutrition_info = []
direction = []
serving = []

for tag in section:
    if tag.name=='h3':
        break
    text=tag.get_text(strip=True)
    if text:
        description.append(text)

# ------------------------------------------------------------------------------------------

    
current = None   # tracks which section we are in
i=0
for tag in section:

    # STEP 1: detect heading
    
    if tag.name == "h3":
        heading = tag.get_text(strip=True).lower()

        if "ingredient" in heading:
            current = "ingredients"
        elif "nutrition" in heading:
            current = "nutrition"
        elif "direction" in heading or "instructions" in heading:
            current = "direction"


        continue   # move to next tag

    # STEP 2: paragraphs
    if tag.name == "p":
        text = tag.get_text(' ',strip=True)
        
        if current == "ingredients":
            ingredients.append(text)
        elif current == "nutrition":
            i+=1
            if i==1:
                serving.append(text)
            if i==2:    
                nutrition_info.append(text)

    # STEP 3: lists (ol / ul)
    if tag.name in ["ul", "ol"]:
        for li in tag.find_all("li"):
            text = li.get_text(' ',strip=True)

            if current == "ingredients":
                ingredients.append(text)
            elif current == "direction":
                direction.append(text) 
direction = ",".join(direction)   # now all the direcions are taken in a string format
if len(ingredients) > 1:
    ingredients=",".join(ingredients)
if len(description) > 1:
    description=",".join(description)    

In [ ]:
print(nutrition_info,'\n')
print(len(nutrition_info))

In [ ]:
print(description[0],'\n')
print(ingredients[0],'\n')
print(direction)   

In [ ]:
print(nutrition)

In [ ]:
print(len(description))
print(len(ingredients))
print(len(nutrition))

In [ ]:
len(breakfast)  # here in breakfast dic we have links of various food items 

In [ ]:
section.ol.get_text(strip=True)  # directions

## From here the main web scrapping is start 

In [ ]:
webpage=requests.get('https://www.eatright.org/recipes').text

In [ ]:
soup=BeautifulSoup(webpage,'lxml')

In [ ]:
categories=soup.find('div',class_='cards-row__grid').find_all('article')

In [ ]:
categories[0].h3.get_text(strip=True)

In [ ]:
len(categories)

https://www.eatright.org

In [ ]:
categories[0].a.get('href')

In [ ]:
for types in categories:
    print(types.h3.get_text(strip=True))
    print(types.a.get('href'),'\n')
    tlink=types.a.get('href')
    url="https://www.eatright.org{}".format(tlink)
    recipes_page=requests.get(url).text
    soup=BeautifulSoup(recipes_page,'lxml')
    a = soup.find('div',class_='article-listing__container').find_all('article')
    for i in a:
        print(i.h3.get_text(strip = True))
        print(i.a.get('href'))
    print('\n')    

In [ ]:
'''for types in categories:
    print(types.h3.get_text(strip=True))
    print(types.a.get('href'),'\n')
    tlink=types.a.get('href')
    url="https://www.eatright.org{}".format(tlink)
    recipes_page=requests.get(url).text
    soup=BeautifulSoup(recipes_page,'lxml')
    a = soup.find('div',class_='article-listing__container').find_all('article')
    for i in a:
        print(i.h3.get_text(strip = True))
        print(i.a.get('href'))
    print('\n') '''

### defining the columns needed in dataset

#####  [ category , recipe_name , description , ingredients , directions , nutrition , servings  ]

In [ ]:
data=[]
count=0
for types in categories:
    category=types.h3.get_text(strip=True)
    tlink=types.a.get('href')
    url = "https://www.eatright.org{}".format(tlink)
    recipes_page = requests.get(url).text
    soup = BeautifulSoup(recipes_page,'lxml')
    a = soup.find('div',class_='article-listing__container').find_all('article')
    for i in a:
        recipe_name = i.h3.get_text(strip = True)
        recipe_link = i.a.get('href')

        rlink = "https://www.eatright.org{}".format(recipe_link)
        recipes= requests.get(rlink).text
        soup2 = BeautifulSoup(recipes,'lxml')
        section=soup2.find("section",class_="rtf rtf--with-margin")
        if section is None:
            continue
        count+=1
        description = []
        ingredients = []
        nutrition_info = []
        direction = []
        serving = []
        
        for tag in section:
            if tag.name=='h3':
                break
            text=tag.get_text(strip=True)
            if text:
                description.append(text)
        
        # ------------------------------------------------------------------------------------------
        
            
        current = None   # tracks which section we are in
        i=0
        for tag in section:
        
            # STEP 1: detect heading
            
            if tag.name == "h3":
                heading = tag.get_text(strip=True).lower()
        
                if "ingredient" in heading:
                    current = "ingredients"
                elif "nutrition" in heading:
                    current = "nutrition"
                elif "direction" in heading or "instructions" in heading:
                    current = "direction"
        
        
                continue   # move to next tag
        
            # STEP 2: paragraphs
            if tag.name == "p":
                text = tag.get_text(' ',strip=True)
                
                if current == "ingredients":
                    ingredients.append(text)
                elif current == "nutrition":
                    i+=1
                    if i==1:
                        serving.append(text)
                    if i==2:    
                        nutrition_info.append(text)
        
            # STEP 3: lists (ol / ul)
            if tag.name in ["ul", "ol"]:
                for li in tag.find_all("li"):
                    text = li.get_text(' ',strip=True)
        
                    if current == "ingredients":
                        ingredients.append(text)
                    elif current == "direction":
                        direction.append(text) 
        direction = ",".join(direction)   # now all the direcions are taken in a string format
        if len(ingredients) > 1:
            ingredients=",".join(ingredients)
        if len(description) > 1:
            description=",".join(description)    
        data.append({
                "category": category,
                "recipe_name": recipe_name,
                "recipe_url": rlink,
                "description":description,
                "ingredients":ingredients,
                "directions": direction,
                "serving": serving,
                "nutrition": nutrition_info
            })
        print(count)
        
    

In [ ]:
len(data)

In [ ]:
print(type(data))
df=pd.DataFrame(data)

In [ ]:
df.sample(6)

In [ ]:
df.to_csv('Web_Scraped_Recipes_Dataset',index=False)